In [1]:
%pip install pandas numpy matplotlib transformers torch scipy rdkit feature-engine scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


### Classical Model Preprocessing


In [2]:
import pandas as pd

# Drop columns with any empty rows
data = pd.read_csv('CycPeptMPDB_Peptide_All.csv', low_memory=False)

In [3]:
print(f"Initial number of rows: {len(data)}")
print(f'# of Columns before dropping: {data.shape[1]}')
data = data.drop_duplicates(subset='Structurally_Unique_ID')
print(f"Number of rows after dropping duplicate molecules: {len(data)}")

Initial number of rows: 8466
# of Columns before dropping: 247
Number of rows after dropping duplicate molecules: 7991


In [4]:
# Remove columns with any missing values
data = data.dropna(axis=1)
print(f"Number of columns after dropping those with missing values: {data.shape[1]}")
print(f"Columns remaining: {data.columns.tolist()}")

# Remove Permeabiltiy = -10
data = data[data['Permeability'] != -10]
print(f"Number of rows after removing Permeability = -10: {len(data)}")


Number of columns after dropping those with missing values: 226
Columns remaining: ['ID', 'Source', 'Year', 'Version', 'Original_Name_in_Source_Literature', 'Structurally_Unique_ID', 'SMILES', 'HELM', 'HELM_URL', 'Sequence', 'Sequence_LogP', 'Sequence_TPSA', 'Monomer_Length', 'Monomer_Length_in_Main_Chain', 'Molecule_Shape', 'Permeability', 'MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', '

In [29]:
# Target and features
y = data['Permeability']
x = data.drop(columns=['Permeability'])

# Drop unnecessary columns
# Identifiers and metadata
x = x.drop(columns=['ID', 'Source', 'Year', 'Version', 'Original_Name_in_Source_Literature', 'Structurally_Unique_ID'])

# SMILES and HELM representations
x = x.drop(columns=['SMILES', 'HELM', 'HELM_URL', 'Sequence'])

# TPSA and Sequence_LogP_Avg already exist in the dataset
x = x.drop(columns=['Sequence_TPSA', 'Sequence_LogP'])

# PC1 and PC2 no documentation available
x = x.drop(columns=['PC1', 'PC2'])

print(f"Final number of features: {x.shape[1]}")
print(f'Remaining features: {x.columns.tolist()}')


Final number of features: 211
Remaining features: ['Monomer_Length', 'Monomer_Length_in_Main_Chain', 'Molecule_Shape', 'MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'S

In [30]:
# One hot encoding for 'Molecule_Shape'
if 'Molecule_Shape' in x.columns:
    print(f"'Molecule_Shape' column found with {x['Molecule_Shape'].nunique()} unique values.")
    x = pd.get_dummies(x, columns=['Molecule_Shape'])
print(f"Number of features after one-hot encoding: {x.shape[1]}")
print(f"Remaining features after encoding: {x.columns.tolist()}")

'Molecule_Shape' column found with 2 unique values.
Number of features after one-hot encoding: 212
Remaining features after encoding: ['Monomer_Length', 'Monomer_Length_in_Main_Chain', 'MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8'

In [35]:
from sklearn.preprocessing import StandardScaler

# Scale numerical features
scaler = StandardScaler()
x_scaled = pd.DataFrame(scaler.fit_transform(x), columns=x.columns, index=x.index)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)

# Train Random Forest Regressor
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

# Evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Range of Permeability in y: {y_scaled.min():.2f} to {y_scaled.max():.2f}")

RMSE: 0.6247
R2: 0.6213
MAE: 0.4554
Range of Permeability in y: -4.68 to 2.40


In [33]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for RandomForestRegressor
param_grid = {
    'n_estimators': [100, 200, 500],  # More trees can reduce variance
    'max_depth': [None, 10, 20, 30],  # Try deeper trees
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [1.0, 'sqrt', 'log2']  # Add feature sampling options
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    error_score='raise'  # Raise an error if any issue occurs during fitting
)

# Fit on training data
grid_search.fit(X_train, y_train)

# Best estimator
best_rf = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_best = best_rf.predict(X_test)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)

print(f"Fine-tuned RMSE: {rmse_best:.4f}")
print(f"Fine-tuned R2: {r2_best:.4f}")
print(f"Fine-tuned MAE: {mae_best:.4f}")

Fitting 3 folds for each of 324 candidates, totalling 972 fits
Best parameters: {'max_depth': 30, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Fine-tuned RMSE: 0.4886
Fine-tuned R2: 0.6252
Fine-tuned MAE: 0.3571


In [34]:
# Print top 20 feature importances from the best Random Forest model after grid search
importances = best_rf.feature_importances_
feature_names = X_train.columns
indices = importances.argsort()[::-1]

print("Top 20 Feature Importances:")
for i in range(20):
    print(f"{feature_names[indices[i]]}: {importances[indices[i]]:.4f}")

Top 20 Feature Importances:
MolLogP: 0.1461
PEOE_VSA6: 0.0446
qed: 0.0420
BCUT2D_CHGLO: 0.0403
BalabanJ: 0.0360
EState_VSA10: 0.0321
fr_Al_OH: 0.0311
BCUT2D_MRLOW: 0.0238
VSA_EState7: 0.0221
BCUT2D_LOGPLOW: 0.0215
MinAbsEStateIndex: 0.0212
VSA_EState2: 0.0172
VSA_EState3: 0.0169
fr_Ndealkylation1: 0.0160
VSA_EState6: 0.0158
SlogP_VSA7: 0.0156
BCUT2D_MWLOW: 0.0152
Kappa3: 0.0150
TPSA: 0.0137
MinPartialCharge: 0.0136
